- 有点乱，大家根据自己习惯瞎改改吧
- kernel替换完成后，不可逆，需要刷新重启
- 测试结果都是取得稳定状态下的迭代速度，大概在20步之后

In [1]:
import os
# os.environ['CUDA_VISIBLE_DEVICES'] = '7'
import torch
import sys
import importlib
# 如果测huggingface中没有模型，需要把路径导进来，不然import了kernel，使用AutoModel加载模型也没有用处
sys.path.append('/mnt/workspace/mdy/models')
module = importlib.import_module('glm-4-9b-chat.modeling_chatglm')
from transformers import AutoConfig, AutoModelForCausalLM, Qwen2ForCausalLM
import time
from copy import deepcopy
from tqdm import tqdm

/mnt/workspace/mdy/miniforge/envs/mdy/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/mnt/workspace/mdy/miniforge/envs/mdy/lib/python3.10/site-packages/_distutils_hack/__init__.py:53: UserWarning: Reliance on distutils from stdlib is deprecated. Users must rely on setuptools to provide the distutils module. Avoid importing distutils or import setuptools first, and avoid setting SETUPTOOLS_USE_DISTUTILS=stdlib. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml
  warnings.warn(


In [2]:

# config = AutoConfig.from_pretrained('/mnt/workspace/mdy/models/Qwen2.5-7B-Instruct')
dtype = torch.bfloat16
bs = 8
seq_len = 2048
iters = 50
model_path = '/mnt/workspace/mdy/models/glm-4-9b-chat'
model_path = '/mnt/workspace/mdy/models/Qwen2.5-0.5B'
# model_path = '/mnt/workspace/mdy/models/Llama-3.2-1B-Instruct'
# model_path = '/mnt/workspace/mdy/models/Meta-Llama-3.1-8B-Instruct'
# model_path = '/mnt/workspace/mdy/models/glm-4-qwen2-format'
model = AutoModelForCausalLM.from_pretrained(model_path,
                                            _attn_implementation='flash_attention_2',
                                           device_map='cuda', torch_dtype=dtype, trust_remote_code=True)
# model = module.ChatGLMForConditionalGeneration.from_pretrained(model_path, _attn_implementation='flash_attention_2',
#                                            device_map='cuda', torch_dtype=dtype, trust_remote_code=True)
input_ids = torch.randint(10000, 20000, (bs, seq_len)).cuda()
pos_ids = torch.arange(seq_len)[None, :].expand(bs, -1).contiguous().cuda()
labels = input_ids
grad_list = []
for p in model.parameters():
    grad_list.append(p)
def zero_gard():
    for p in grad_list:
        if p.grad is not None:
            p.grad.zero_()


In [4]:
torch.cuda.empty_cache()
t = time.time()
for i in tqdm(range(iters)):
    out = model(input_ids, position_ids=pos_ids, labels=labels)
    out.loss.backward()
t1 = time.time() - t
t1

100%|██████████| 50/50 [00:26<00:00,  1.92it/s]


26.03295111656189

In [5]:
print(grad_list[10].grad[:5])
zero_gard()

tensor([[ 0.0201,  0.0309, -0.0640,  ..., -0.0102,  0.0223, -0.0292],
        [-0.0045, -0.0060, -0.0243,  ...,  0.0200, -0.0092,  0.0042],
        [ 0.0060,  0.0205, -0.0488,  ...,  0.0082,  0.0026, -0.0102],
        [ 0.0003, -0.0130,  0.0079,  ...,  0.0103, -0.0087, -0.0079],
        [-0.0061, -0.0150, -0.0076,  ...,  0.0011,  0.0073, -0.0015]],
       device='cuda:0', dtype=torch.bfloat16)


In [6]:
from mdy_triton.replace_kernel import *

In [7]:
torch.cuda.empty_cache()
t = time.time()
for i in tqdm(range(iters)):
    out = model(input_ids, position_ids=pos_ids,labels=labels)
    out.loss.backward()
t2 = time.time() - t
t2

100%|██████████| 50/50 [00:16<00:00,  3.11it/s]


16.084028959274292

In [8]:
print(grad_list[10].grad[:5])
zero_gard()

tensor([[ 0.0204,  0.0309, -0.0640,  ..., -0.0098,  0.0226, -0.0302],
        [-0.0046, -0.0063, -0.0242,  ...,  0.0200, -0.0092,  0.0046],
        [ 0.0054,  0.0201, -0.0488,  ...,  0.0090,  0.0029, -0.0101],
        [ 0.0003, -0.0130,  0.0079,  ...,  0.0112, -0.0090, -0.0081],
        [-0.0065, -0.0146, -0.0073,  ...,  0.0012,  0.0071, -0.0016]],
       device='cuda:0', dtype=torch.bfloat16)


# loss显存占用

In [2]:
torch.cuda.empty_cache()
bs = 8
seq_len = 2048
vocab_size = 150000
dtype=torch.bfloat16
factor = 4 if dtype == torch.float32 else 2
print('logits显存占用：',(bs * seq_len * vocab_size) / (1024)**3 * factor,"G")
logits = torch.randn(bs, seq_len, vocab_size, device='cuda', dtype=dtype, requires_grad=True)
labels = torch.randint(0, 10, (bs, seq_len)).cuda()
for i in tqdm(range(200)):
    logits = logits.float()
    shift_logits = logits[:, :-1].contiguous()
    shift_labels = labels[:, 1:].contiguous()
    loss_fn = torch.nn.CrossEntropyLoss()
    loss = loss_fn(shift_logits.view(-1, vocab_size), shift_labels.view(-1))
    loss.backward()
    # break
# 0.12s/it

logits显存占用： 4.57763671875 G


100%|██████████| 200/200 [00:21<00:00,  9.15it/s]


In [ ]:
from mdy_triton.core import fast_cross_entropy_loss
torch.cuda.empty_cache()
bs = 8
seq_len = 2048
vocab_size = 150000
dtype=torch.bfloat16
factor = 4 if dtype == torch.float32 else 2
print('logits显存占用：',(bs * seq_len * vocab_size) / (1024)**3 * factor,"G")
extra_ignored_labels = torch.full((seq_len, 1), -100, device='cuda')
logits = torch.randn(bs, seq_len, vocab_size, device='cuda', dtype=dtype, requires_grad=True)
labels = torch.randint(0, vocab_size-1, (bs, seq_len)).cuda()
for i in tqdm(range(200)):
    shift_logits = logits
    shift_labels = torch.hstack((labels[..., 1:], extra_ignored_labels[:labels.shape[0]]))
    loss = fast_cross_entropy_loss(shift_logits, shift_labels)
    loss.backward()
    # break
# 重启运行
# 0.02s/it

logits显存占用： 4.57763671875 G


100%|██████████| 200/200 [00:03<00:00, 53.64it/s]
